In [1]:
import nltk
import re

In [2]:
paragraph = """
Chief Bola Ahmed Adekunle Tinubu GCFR (born 29 March 1952) is a Nigerian politician who is the 16th and current president of Nigeria.[1] He was the governor of Lagos State from 1999 to 2007; and senator for Lagos West in the Third Republic.

Tinubu spent his early life in southwestern Nigeria and later moved to the United States where he studied accounting at Chicago State University. He returned to Nigeria in the early 1990s and was employed by Mobil Nigeria as an accountant, before entering politics as a Lagos West senatorial candidate in 1992 under the banner of the Social Democratic Party. After dictator Sani Abacha dissolved the Senate in 1993, Tinubu became an activist campaigning for the return of democracy as a part of the National Democratic Coalition movement.

In the first post-transition Lagos State gubernatorial election, Tinubu won by a wide margin as a member of the Alliance for Democracy. Four years later, he won re-election to a second term. After leaving office in 2007, he played a key role in the formation of the All Progressives Congress in 2013. In 2023, he was elected president of Nigeria.
"""

In [3]:
paragraph

'\nChief Bola Ahmed Adekunle Tinubu GCFR (born 29 March 1952) is a Nigerian politician who is the 16th and current president of Nigeria.[1] He was the governor of Lagos State from 1999 to 2007; and senator for Lagos West in the Third Republic.\n\nTinubu spent his early life in southwestern Nigeria and later moved to the United States where he studied accounting at Chicago State University. He returned to Nigeria in the early 1990s and was employed by Mobil Nigeria as an accountant, before entering politics as a Lagos West senatorial candidate in 1992 under the banner of the Social Democratic Party. After dictator Sani Abacha dissolved the Senate in 1993, Tinubu became an activist campaigning for the return of democracy as a part of the National Democratic Coalition movement.\n\nIn the first post-transition Lagos State gubernatorial election, Tinubu won by a wide margin as a member of the Alliance for Democracy. Four years later, he won re-election to a second term. After leaving office

In [16]:
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [6]:
# tokenization

nltk.download('punkt')
sentences = nltk.sent_tokenize(paragraph)

[nltk_data] Downloading package punkt to C:\Users\ACER-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [12]:
stemmer = PorterStemmer()

In [15]:
lemmatize = WordNetLemmatizer()

In [17]:
corpus = []
for i in range(len(sentences)):
    review = re.sub('[^a-zA-Z]', ' ', sentences[i])
    review=review.lower()
    corpus.append(review)

In [19]:

corpus

[' chief bola ahmed adekunle tinubu gcfr  born    march       is a nigerian politician who is the   th and current president of nigeria ',
 '    he was the governor of lagos state from      to       and senator for lagos west in the third republic ',
 'tinubu spent his early life in southwestern nigeria and later moved to the united states where he studied accounting at chicago state university ',
 'he returned to nigeria in the early     s and was employed by mobil nigeria as an accountant  before entering politics as a lagos west senatorial candidate in      under the banner of the social democratic party ',
 'after dictator sani abacha dissolved the senate in       tinubu became an activist campaigning for the return of democracy as a part of the national democratic coalition movement ',
 'in the first post transition lagos state gubernatorial election  tinubu won by a wide margin as a member of the alliance for democracy ',
 'four years later  he won re election to a second term ',